In [1]:
import torch
from monai.transforms import ScaleIntensityRanged
from generate_transforms import generate_detection_inference_transform
from monai.data import DataLoader, Dataset, load_decathlon_datalist
from monai.data.utils import no_collation

In [2]:
amp = True
# gt_box_mode = "cccwhd"
# src_mode = "xyzxyz"
gt_box_mode = "xyzxyz"
src_mode = "cccwhd"
data_list_file_path = '/data/output/LUNA16-micro.json'
data_base_dir = "/data/LUNA16_Images_resample/"

In [3]:
# 1. define transform
intensity_transform = ScaleIntensityRanged(
    keys=["image"],
    a_min=-1024,
    a_max=300.0,
    b_min=0.0,
    b_max=1.0,
    clip=True,
)

inference_transforms, post_transforms = generate_detection_inference_transform(
    "image",
    "box",
    "pred_label",
    "pred_score",
    gt_box_mode,
    src_mode,
    intensity_transform,
    affine_lps_to_ras=True,
    amp=amp,
)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


In [9]:
# 2. create a inference data loader
inference_data = load_decathlon_datalist(
    data_list_file_path,
    is_segmentation=True,
    data_list_key="training",
    base_dir=data_base_dir,
)

inference_ds = Dataset(
    data=inference_data,
    transform=inference_transforms,
)

inference_loader = DataLoader(
    inference_ds,
    batch_size=1,
    pin_memory=torch.cuda.is_available(),
    collate_fn=no_collation,
)

In [10]:
for inference_data in inference_loader:
    for inference_data_i in inference_data:
        # print(inference_data_i['image_meta_dict'])
        # print(inference_data_i['image_meta_dict'].keys())
        print(inference_data_i['box'], inference_data_i['image_meta_dict']['filename_or_obj'])

[[100.0021827, 22.00611876, -147.0757546, 8.871621958, 8.871621958, 8.871621958], [-55.31574942, 4.042374673, -154.7552924, 5.970538617, 5.970538617, 5.970538617], [-25.40860122, 136.715217, -181.6943198, 11.08601237, 11.08601237, 11.08601237], [127.2115066, 44.79847847, -164.7894133, 7.72427841, 7.72427841, 7.72427841], [120.2208995, 50.312055, -132.1372108, 5.251592099, 5.251592099, 5.251592099], [101.082909, -3.953008029, -180.4023007, 9.404294354, 9.404294354, 9.404294354]] /data/LUNA16_Images_resample/1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143/1.3.6.1.4.1.14519.5.2.1.6279.6001.250397690690072950000431855143.nii.gz
[[102.2377292, 65.32842988, -240.6955559, 18.210145, 18.210145, 18.210145], [22.53616818, 34.72145001, -117.4375995, 10.33971307, 10.33971307, 10.33971307], [60.18947724, -45.04675978, -112.6427797, 8.349225392, 8.349225392, 8.349225392], [67.82725575, 85.37992457, -109.7467238, 32.27003025, 32.27003025, 32.27003025], [114.1058249, -11.85681073, -16

In [6]:
[[100.0021827, 22.00611876, -147.0757546, 8.871621958, 8.871621958, 8.871621958], 
 [-55.31574942, 4.042374673, -154.7552924, 5.970538617, 5.970538617, 5.970538617], 
 [-25.40860122, 136.715217, -181.6943198, 11.08601237, 11.08601237, 11.08601237], 
 [127.2115066, 44.79847847, -164.7894133, 7.72427841, 7.72427841, 7.72427841], 
 [120.2208995, 50.312055, -132.1372108, 5.251592099, 5.251592099, 5.251592099], 
 [101.082909, -3.953008029, -180.4023007, 9.404294354, 9.404294354, 9.404294354]]

[[100.0021827,
  22.00611876,
  -147.0757546,
  8.871621958,
  8.871621958,
  8.871621958],
 [-55.31574942,
  4.042374673,
  -154.7552924,
  5.970538617,
  5.970538617,
  5.970538617],
 [-25.40860122,
  136.715217,
  -181.6943198,
  11.08601237,
  11.08601237,
  11.08601237],
 [127.2115066, 44.79847847, -164.7894133, 7.72427841, 7.72427841, 7.72427841],
 [120.2208995, 50.312055, -132.1372108, 5.251592099, 5.251592099, 5.251592099],
 [101.082909,
  -3.953008029,
  -180.4023007,
  9.404294354,
  9.404294354,
  9.404294354]]

In [7]:
inference_data_i.keys()

dict_keys(['box', 'image', 'label', 'image_meta_dict'])

In [8]:
inference_data_i['image_meta_dict']

{'sizeof_hdr': array(348, dtype=int32),
 'extents': array(0, dtype=int32),
 'session_error': array(0, dtype=int16),
 'dim_info': array(0, dtype=uint8),
 'dim': array([  3, 512, 512, 245,   1,   1,   1,   1], dtype=int16),
 'intent_p1': array(0., dtype=float32),
 'intent_p2': array(0., dtype=float32),
 'intent_p3': array(0., dtype=float32),
 'intent_code': array(0, dtype=int16),
 'datatype': array(16, dtype=int16),
 'bitpix': array(32, dtype=int16),
 'slice_start': array(0, dtype=int16),
 'pixdim': array([1.      , 0.703125, 0.703125, 1.25    , 1.      , 1.      ,
        1.      , 1.      ], dtype=float32),
 'vox_offset': array(0., dtype=float32),
 'scl_slope': array(nan, dtype=float32),
 'scl_inter': array(nan, dtype=float32),
 'slice_end': array(0, dtype=int16),
 'slice_code': array(0, dtype=uint8),
 'xyzt_units': array(0, dtype=uint8),
 'cal_max': array(0., dtype=float32),
 'cal_min': array(0., dtype=float32),
 'slice_duration': array(0., dtype=float32),
 'toffset': array(0., dtype=